In [1]:
import pandas as pd
import random
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import time
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

/home/veruska/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
city = 'Recife'

In [3]:
data_path = '../../data/output/' + city + '/normalized_data_X.csv'
y_path = '../../data/output/' + city + '/y.csv'

data = pd.read_csv(data_path)
y = pd.read_csv(y_path)
print(data.shape)
print(y.shape)
data.head()

(1432633, 420)
(1432633, 1)


,route,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,...,alertTypeSB_ACCIDENT,alertTypeSB_CHIT_CHAT,alertTypeSB_HAZARD,alertTypeSB_JAM,alertTypeSB_NORMAL,alertTypeSB_POLICE,alertTypeSB_ROAD_CLOSED,jamBlockTypeSB_-,jamBlockTypeSB_NORMAL,jamBlockTypeSB_ROAD_CLOSED_EVENT
0,0.118068,0.066667,0.085019,0.264600,0.496150,0.057584,0.0,0.000000,0.266155,0.496047,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.118068,0.066667,0.085026,0.270218,0.501802,0.064231,0.0,0.000148,0.271812,0.501642,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.118068,0.066667,0.085033,0.273653,0.505296,0.068301,0.0,0.000000,0.275187,0.505192,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.118068,0.066667,0.085039,0.277433,0.509404,0.072900,0.0,0.000149,0.280622,0.510132,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.118068,0.066667,0.085053,0.280050,0.513166,0.077983,0.0,0.000155,0.281878,0.513178,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
#Bus bunching proportion per data
len(y[y.busBunching == 1]) / len(y)

0.13544362024328632

In [13]:
# same data of rvm test
data = data[0:1000]
y = y[0:1000]
print(data.shape)
print(y.shape)

(1000, 420)
(1000, 1)


In [4]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y, test_size=0.20, random_state=7)

In [3]:
test_X = pd.read_csv('../../data/output/' + city + '/test_X.csv')
test_Y = pd.read_csv('../../data/output/' + city + '/test_Y.csv')

In [5]:
del y
del data

In [ ]:
random.seed(42)

start = time.time()

parameters = {
    'max_depth': [5, 25],
    'n_estimators': [50, 100],
    'learning_rate': [0.1, 0.01]
}

xgb = XGBClassifier(
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 #nthread=-1,
 scale_pos_weight=1,
 seed=23)

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=parameters,
    scoring = 'roc_auc',
    cv = 10
)

grid_search.fit(train_X, train_Y)

end = time.time()
print("Execution time: " + str((end - start)/60) + " min")

In [6]:
random.seed(42)

start = time.time()

xgb = XGBClassifier(
 learning_rate =0.1,
 n_estimators=120, 
 max_depth=50,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=1,
 seed=23)

xgb.fit(train_X, train_Y)

end = time.time()
print("Execution time: " + str((end - start)/60) + " min")

/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Execution time: 31.07039443254471 min


In [10]:
print("Execution time: " + str((end - start)/60) + " min") #all - Curitiba

Execution time: 1399.5225323120753 min


In [10]:
# Saving a pickle file for the mode
# joblib.dump(xgb, 'Saved_XGB_BB_class.pkl')
#joblib.dump(xgb, 'Saved_Curitiba_XGB_BB_class.pkl')
#joblib.dump(xgb, 'Saved_Recife_XGB_75p_BB_class.pkl')
#joblib.dump(xgb, 'Saved_Recife_XGB_52p_BB_class.pkl')
#joblib.dump(xgb, 'Saved_Recife_XGB_25p_BB_class.pkl')
joblib.dump(xgb, 'Saved_Recife_XGB_BB_class.pkl')

['Saved_Recife_XGB_BB_class.pkl']

In [7]:
pred_array = xgb.predict(test_X)
pred_array

array([0., 1., 0., ..., 0., 0., 0.])

In [8]:
# Bus Bunching
accuracy = accuracy_score(test_Y, pred_array)
precision = precision_score(test_Y, pred_array)
recall = recall_score(test_Y, pred_array)
f_measure = f1_score(test_Y, pred_array)

In [11]:
# all data - Curitiba
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9795285249944861
Precision: 0.8751984757065735
Recall: 0.7445577215657837
F-measure: 0.8046097111247695


In [10]:
# gps+gtfs - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9510098524746359
Precision: 0.9056198347107438
Recall: 0.7101381652279856
F-measure: 0.7960538742063434


In [10]:
# gps+gtfs+weather - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9508178984877516
Precision: 0.9070656691604323
Recall: 0.7071571143427431
F-measure: 0.794732855561382


In [13]:
# gps+gtfs+waze - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9502734471794979
Precision: 0.9060382522781134
Recall: 0.7036316976436737
F-measure: 0.7921092564491654


In [7]:
# 5p - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.8688675063781075
Precision: 0.585432266848196
Recall: 0.08917230474116701
F-measure: 0.15477020673520347


In [11]:
# 25p - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.8899196236305829
Precision: 0.767772872583346
Recall: 0.26147704590818366
F-measure: 0.39009958425988595


In [15]:
# 52p - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9328265748079587
Precision: 0.8834404506863445
Recall: 0.5772351401093916
F-measure: 0.6982424784033363


In [19]:
# 75p - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9560913980183369
Precision: 0.944923666735127
Recall: 0.7155818233662545
F-measure: 0.8144148928323819


In [18]:
# 1000, to compare with rvm
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.975
Precision: 1.0
Recall: 0.5454545454545454
F-measure: 0.7058823529411764


In [9]:
# 3000, to compare with rvm
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9733333333333334
Precision: 0.9634146341463414
Recall: 0.8586956521739131
F-measure: 0.9080459770114943


In [9]:
# all data - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9502525067445651
Precision: 0.9080629466832199
Recall: 0.7015320009331986
F-measure: 0.7915472360339281


In [9]:
# all data - Recife
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9502525067445651
Precision: 0.9080629466832199
Recall: 0.7015320009331986
F-measure: 0.7915472360339281
